In [1]:
import pandas as pd
import re
import pickle
pd.set_option('display.max_columns', None)
import sys
!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
filename = 'all_data_anonymized.csv'
df_read = pd.read_csv(filename, sep=';')
df_read.head()

C:\Users\oscar.azrak\AppData\Local\Temp\ipykernel_15892\2576158672.py:2: DtypeWarning: Columns (15,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_read = pd.read_csv(filename, sep=';')


,todate,EQ_0,EQ_1,EQ_2,EQ_3,EQ_4,EQ_5,EQ_6,EQ_7,EQ_8,EQ_9,EQ_10,EQ_11,EQ_12,EQ_13,EQ_14,EQ_15,EQ_16,EQS_0,EQS_1,EQS_2,EQS_3,EQS_4,EQS_5,EQS_6,EQS_7,EQS_8,EQS_9,EQS_10,FI_0,FI_1,FI_2,FI_3,FI_4,FI_5,FI_6,FI_7,FI_8,FI_9,FI_10,FI_11,FI_12,FI_13,FI_14,FXD_0,FXD_1,FXD_2,FXD_3,FXD_4,FXD_5,FXD_6,FXD_7,FXD_8,FXD_9,FXE_0,FXE_1,FXE_2,FXE_3,FXE_4,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
0,1970-01-05 00:00:00,NaN,"0,004909044",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,00595314",NaN,"0,006268081","0,031211244",NaN,"-0,013558163"
1,1970-01-06 00:00:00,NaN,"-0,006821403",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,002578518",NaN,"-0,026832774","0,030467027",NaN,"0,004273504"
2,1970-01-07 00:00:00,NaN,"-0,002124071",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000843241",NaN,"-0,021664205","0,022758218",NaN,"0,005980449"
3,1970-01-08 00:00:00,NaN,"0,000564636",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,003370124",NaN,"0,014594867","0,031951312",NaN,"-0,000914599"
4,1970-01-09 00:00:00,NaN,"-0,003075721",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-0,005919893",NaN,"-0,013392857","-0,003133063",NaN,"-0,007552352"


In [3]:

# set date to index
dateCol = 'todate'

df_read[dateCol] = pd.to_datetime(df_read[dateCol])
df_read.set_index(dateCol, inplace=True)


for column in df_read.columns:
    # Replace commas with periods and convert to float
    df_read[column] = df_read[column].str.replace(',', '.').astype(float)
df_read.head()

,EQ_0,EQ_1,EQ_2,EQ_3,EQ_4,EQ_5,EQ_6,EQ_7,EQ_8,EQ_9,EQ_10,EQ_11,EQ_12,EQ_13,EQ_14,EQ_15,EQ_16,EQS_0,EQS_1,EQS_2,EQS_3,EQS_4,EQS_5,EQS_6,EQS_7,EQS_8,EQS_9,EQS_10,FI_0,FI_1,FI_2,FI_3,FI_4,FI_5,FI_6,FI_7,FI_8,FI_9,FI_10,FI_11,FI_12,FI_13,FI_14,FXD_0,FXD_1,FXD_2,FXD_3,FXD_4,FXD_5,FXD_6,FXD_7,FXD_8,FXD_9,FXE_0,FXE_1,FXE_2,FXE_3,FXE_4,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
todate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1970-01-05,NaN,0.004909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005953,NaN,0.006268,0.031211,NaN,-0.013558
1970-01-06,NaN,-0.006821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002579,NaN,-0.026833,0.030467,NaN,0.004274
1970-01-07,NaN,-0.002124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000843,NaN,-0.021664,0.022758,NaN,0.005980
1970-01-08,NaN,0.000565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003370,NaN,0.014595,0.031951,NaN,-0.000915
1970-01-09,NaN,-0.003076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.005920,NaN,-0.013393,-0.003133,NaN,-0.007552


In [4]:
startdate = '1980-01-01'

df = df_read[startdate:]
df.head()

,EQ_0,EQ_1,EQ_2,EQ_3,EQ_4,EQ_5,EQ_6,EQ_7,EQ_8,EQ_9,EQ_10,EQ_11,EQ_12,EQ_13,EQ_14,EQ_15,EQ_16,EQS_0,EQS_1,EQS_2,EQS_3,EQS_4,EQS_5,EQS_6,EQS_7,EQS_8,EQS_9,EQS_10,FI_0,FI_1,FI_2,FI_3,FI_4,FI_5,FI_6,FI_7,FI_8,FI_9,FI_10,FI_11,FI_12,FI_13,FI_14,FXD_0,FXD_1,FXD_2,FXD_3,FXD_4,FXD_5,FXD_6,FXD_7,FXD_8,FXD_9,FXE_0,FXE_1,FXE_2,FXE_3,FXE_4,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
todate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1980-01-02,NaN,-0.018988,NaN,NaN,NaN,NaN,NaN,NaN,-0.018316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.046210,NaN,0.010604,NaN,-0.005429,-0.041021,-0.021920,-0.019289
1980-01-03,NaN,-0.005294,NaN,NaN,NaN,NaN,NaN,NaN,-0.015509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.004372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.044173,NaN,0.001396,NaN,0.002396,-0.017317,-0.034213,0.001675
1980-01-04,NaN,0.012279,NaN,NaN,NaN,NaN,NaN,NaN,0.017255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.004391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.033839,NaN,-0.014684,NaN,0.006773,0.028012,0.027159,-0.001672
1980-01-07,NaN,0.002119,NaN,NaN,NaN,NaN,NaN,NaN,0.005529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.037644,NaN,-0.021268,NaN,0.009445,-0.026609,0.013745,0.000000
1980-01-08,NaN,0.019638,NaN,NaN,NaN,NaN,NaN,NaN,0.015610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.039435,NaN,-0.015227,NaN,0.012703,0.005257,0.011143,0.000000


In [5]:
# backfill nan values
# ändra till 0 och limit 5
df = df.fillna(0, limit=5)


In [6]:
# only keep date and columns including "EQ_" and "MACRO_"
#df = df.filter(regex='EQ_|MACRO_')


df.head(20)


,EQ_0,EQ_1,EQ_2,EQ_3,EQ_4,EQ_5,EQ_6,EQ_7,EQ_8,EQ_9,EQ_10,EQ_11,EQ_12,EQ_13,EQ_14,EQ_15,EQ_16,EQS_0,EQS_1,EQS_2,EQS_3,EQS_4,EQS_5,EQS_6,EQS_7,EQS_8,EQS_9,EQS_10,FI_0,FI_1,FI_2,FI_3,FI_4,FI_5,FI_6,FI_7,FI_8,FI_9,FI_10,FI_11,FI_12,FI_13,FI_14,FXD_0,FXD_1,FXD_2,FXD_3,FXD_4,FXD_5,FXD_6,FXD_7,FXD_8,FXD_9,FXE_0,FXE_1,FXE_2,FXE_3,FXE_4,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
todate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1980-01-02,0.0,-0.018988,0.0,0.0,0.0,0.0,0.0,0.0,-0.018316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046210,0.0,0.010604,0.0,-0.005429,-0.041021,-0.021920,-0.019289
1980-01-03,0.0,-0.005294,0.0,0.0,0.0,0.0,0.0,0.0,-0.015509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004372,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.044173,0.0,0.001396,0.0,0.002396,-0.017317,-0.034213,0.001675
1980-01-04,0.0,0.012279,0.0,0.0,0.0,0.0,0.0,0.0,0.017255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033839,0.0,-0.014684,0.0,0.006773,0.028012,0.027159,-0.001672
1980-01-07,0.0,0.002119,0.0,0.0,0.0,0.0,0.0,0.0,0.005529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037644,0.0,-0.021268,0.0,0.009445,-0.026609,0.013745,0.000000
1980-01-08,0.0,0.019638,0.0,0.0,0.0,0.0,0.0,0.0,0.015610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.039435,0.0,-0.015227,0.0,0.012703,0.005257,0.011143,0.000000
1980-01-09,NaN,0.000867,NaN,NaN,NaN,NaN,NaN,NaN,0.006111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.006568,NaN,-0.003696,NaN,-0.006607,0.036214,-0.024429,-0.044877
1980-01-10,NaN,0.006898,NaN,NaN,NaN,NaN,NaN,NaN,0.011597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.003304,NaN,0.009246,NaN,-0.002312,-0.013121,0.018943,-0.024100
1980-01-11,NaN,0.001048,NaN,NaN,NaN,NaN,NaN,NaN,0.005513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.014247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.041458,NaN,-0.019756,NaN,0.021640,0.054973,0.019621,0.022898
1980-01-14,NaN,0.004376,NaN,NaN,NaN,NaN,NaN,NaN,0.008675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.003317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.039808,NaN,-0.010835,NaN,-0.008488,0.048594,-0.022082,0.004684


In [7]:
import numpy as np
windows = [5, 10, 20, 40, 60, 100, 180, 240, 360, 480]
window_m = [10, 30, 60, 100, 180]
# skicka in window som inoput ist

assets = df.columns


In [8]:
feature_cols = [col for col in df.columns if 'macro' not in col.lower()]
macro_cols = [col for col in df.columns if 'MACRO' in col.upper()]

In [9]:


def add_features(df, windows, feature_cols=feature_cols, macro_cols=macro_cols):
    # Initialize a dictionary to hold all the new feature data
    features_dict = {}


    # Perform rolling calculations for each window size
    for w in windows:
        for col in feature_cols:
            # Create unique feature names for each statistic and window size
            features_dict[f'{col}_VaR_{w}'] = df[col].rolling(window=w, min_periods=int(w//2)).quantile(0.05)
            features_dict[f'{col}_momentum_{w}'] = df[col].rolling(window=w, min_periods=int(w//2)).sum()
            features_dict[f'{col}_avgreturn_{w}'] = df[col].rolling(window=w, min_periods=int(w//2)).mean()
            features_dict[f'{col}_skew_{w}'] = df[col].rolling(window=w, min_periods=int(w//2)).skew()
            features_dict[f'{col}_volatility_{w}'] = df[col].rolling(window=w, min_periods=int(w//2)).std()

    # Convert the dictionary of Series to a DataFrame
    features_df = pd.DataFrame(features_dict, index=df.index)

    # Concatenate 'MACRO' columns to the features DataFrame
    if macro_cols:
        macro_df = df[macro_cols]
        features_df = pd.concat([features_df, macro_df], axis=1)

    return features_df


df = add_features(df, window_m)


In [10]:
# Reset the index to make it a regular column
df_reset = df.reset_index()

# Now, you can specify the former index column (now a regular column) as the id_vars
long_df = df_reset.melt(id_vars=dateCol, var_name='metric', value_name='value')



In [11]:

# split 'metric' column and expand to new DataFrame
split_metrics = long_df['metric'].str.split('_', expand=True)

# Save Macro rows
macro_mask = split_metrics[0] == 'MACRO'

# For non-'MACRO' metrics, construct 'asset' by concatenating the first two parts, and the rest as 'metric_type'
long_df['asset'] = split_metrics[0] + '_' + split_metrics[1]
long_df['metric_type'] = split_metrics[2]+ '_' + split_metrics[3]


# added due to previous error
# Reset index on the left-hand side DataFrame slice to ensure alignment
lhs = long_df.loc[macro_mask, 'metric_type'].reset_index(drop=True)

# Reset index on the right-hand side Series to ensure alignment
rhs = (split_metrics.loc[macro_mask, 0] + '_' + split_metrics.loc[macro_mask, 1]).reset_index(drop=True)

# Assign the values after ensuring both sides have the same length
lhs = rhs

# Assign the modified Series back to the original DataFrame (if needed)
long_df.loc[macro_mask, 'metric_type'] = lhs.values

# For 'MACRO' metrics, set 'metric_type' directly as the metric name, and use 'MACRO' as a placeholder in 'asset'
long_df.loc[macro_mask, 'metric_type'] = split_metrics.loc[macro_mask, 0] + '_' + split_metrics.loc[macro_mask, 1]
long_df.loc[macro_mask, 'asset'] = 'MACRO'

# Remove 'MACRO' placeholder rows to avoid 'MACRO' appearing in the 'asset' column of the final DataFrame
long_df = long_df[long_df['asset'] != 'MACRO']

# Pivot the DataFrame to wide format, with 'date' and 'asset' as indexes, and 'metric_type' as columns
final_df = long_df.pivot_table(index=[dateCol, 'asset'], columns='metric_type', values='value').reset_index()

# Incorporate 'MACRO' metrics as separate columns
# Since 'MACRO' metrics do not depend on 'asset', we can directly merge them on 'todate'
macro_df = df.filter(regex='^MACRO').copy()
macro_df[dateCol] = df_reset[dateCol]


In [12]:
if dateCol in macro_df.columns:
    macro_df = macro_df.drop(columns=dateCol)

# Reset the index to turn the 'todate' index into a column
macro_df = macro_df.reset_index()


In [13]:
final_df = pd.merge(final_df, macro_df, on=dateCol, how='left')

In [14]:
# Display the first few rows of the transformed DataFrame
final_df.tail()

,todate,asset,VaR_10,VaR_100,VaR_180,VaR_30,VaR_60,avgreturn_10,avgreturn_100,avgreturn_180,avgreturn_30,avgreturn_60,momentum_10,momentum_100,momentum_180,momentum_30,momentum_60,skew_10,skew_100,skew_180,skew_30,skew_60,volatility_10,volatility_100,volatility_180,volatility_30,volatility_60,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
516689,2024-01-24,FXE_0,-0.013816,-0.013255,-0.013552,-0.014070,-0.012926,-0.001469,0.000110,0.000037,-0.000268,0.000241,-0.011753,0.010443,0.006360,-0.006972,0.013259,-0.954557,-0.252313,-0.277540,-0.510591,-0.266288,0.007897,0.006779,0.007056,0.007212,0.007058,0.077863,-0.004837,0.009681,0.004011,0.00048,0.032585,0.029895,-0.018139,0.023889
516690,2024-01-24,FXE_1,-0.015158,-0.013868,-0.013179,-0.008364,-0.012319,-0.002559,0.000263,0.000290,0.000064,0.000865,-0.020470,0.024990,0.049641,0.001658,0.047560,-1.097757,-0.294224,-0.402596,-1.419161,-0.532774,0.008292,0.007708,0.007083,0.005868,0.007175,0.077863,-0.004837,0.009681,0.004011,0.00048,0.032585,0.029895,-0.018139,0.023889
516691,2024-01-24,FXE_2,-0.013177,-0.014512,-0.014599,-0.015306,-0.014512,-0.001711,0.000200,0.000152,-0.000095,-0.000268,-0.013687,0.019007,0.025926,-0.002474,-0.014715,-0.306785,0.168480,-0.094806,0.218538,0.398629,0.008352,0.008911,0.009385,0.009693,0.009432,0.077863,-0.004837,0.009681,0.004011,0.00048,0.032585,0.029895,-0.018139,0.023889
516692,2024-01-24,FXE_3,-0.008017,-0.014883,-0.017286,-0.015107,-0.015367,0.000754,0.001602,-0.000022,0.001005,0.001471,0.006030,0.152231,-0.003820,0.026138,0.080896,0.557180,0.469147,0.431342,0.579726,0.547749,0.007864,0.013486,0.012715,0.010225,0.015388,0.077863,-0.004837,0.009681,0.004011,0.00048,0.032585,0.029895,-0.018139,0.023889
516693,2024-01-24,FXE_4,-0.002425,-0.001679,-0.002679,-0.001829,-0.001663,-0.000249,0.000008,-0.000034,0.000054,0.000016,-0.002242,0.000755,-0.006037,0.001506,0.000922,-1.035707,0.174639,-0.195346,0.217222,0.278325,0.001381,0.001073,0.001514,0.001376,0.001086,0.077863,-0.004837,0.009681,0.004011,0.00048,0.032585,0.029895,-0.018139,0.023889


In [15]:


with open('final_df_m.pkl', 'wb') as f:
    pickle.dump(final_df, f)
    f.close()


In [16]:
# save to csv
#final_df.to_csv('final_df_m.csv', index=False)

In [17]:
# only keep assets in list
keep_assets = []

# if keep assets is empty, keep all assets otherwise only keep the assets from the list

if keep_assets:
    final_df = final_df[final_df['asset'].str.contains('|'.join(keep_assets))]
else:
    pass




In [18]:
kept_assets = final_df['asset'].unique().tolist()
if len(kept_assets) != 58:
    print('Amount of assets are: ' + str(len(kept_assets)))
else:
    print('All assets are kept')

All assets are kept


Här börjar vi med ML modellen

In [19]:
# RIDGE REGRESSION


RIDGE - 
LIGHTGBM - RANDOM FOREST, GRADIENT BOOST, DESICION TREES (PARAMETER FÖR RF)
ARIMA/TSA
TUNING: splitta 50/50 för tuning, 50 för validering, valideras mha accuracy
        Splitta i tid , train test o val i tid, mycket träning, fram tills 2000, val: 5-års

målfunktion: 4 veckor , sharpe (avkastning som modulärt) y (sharpe efter normalisering) ska ha -1 på de sämsta, 1 på de bästa
np.sign
x-normalisering, använda z-norm
(summa på avkstning)/vol
minperiods = 12
bra för test/exjobbet: predicta med framtidens data

shifta bak: (skifta period +- 1)


### Start by creating the Y-column for the model

In [20]:
#Start by creating the Y-column of the ML model
#vol10 is standard deviation for 10 days
#avgreturn10 is average return for 10 days
#variabel på target dagar

target_days = 9

return_column_shift = 'avgreturn_10'
volatility_column_shift = 'volatility_10'

#create new column by shifting avgreturn10 by 1 day
final_df['avgreturn10_shifted'] = final_df.groupby('asset')[return_column_shift].shift(-target_days)
final_df['volatility_10_shifted'] = final_df.groupby('asset')[volatility_column_shift].shift(-target_days)

#drop rows with NaN
final_df = final_df.dropna()

#create sharpe ratio column by deviding avgreturn10_shifted by vol10
final_df['sharpe_ratio'] = final_df['avgreturn10_shifted'] / final_df['volatility_10_shifted']

#calculate the mean sharpe ratio for each todate
sharpe_ratio_mean = final_df.groupby(dateCol)['sharpe_ratio'].mean()

#add the mean sharpe ratio to the final_df
final_df = final_df.merge(sharpe_ratio_mean, on=dateCol, suffixes=('', '_mean'))

#create new column Y, 1 if sharpe ratio is higher than the mean, 0 otherwise
final_df['Y'] = np.where(final_df['sharpe_ratio'] > final_df['sharpe_ratio_mean'], 1, 0)

final_df.head()

,todate,asset,VaR_10,VaR_100,VaR_180,VaR_30,VaR_60,avgreturn_10,avgreturn_100,avgreturn_180,avgreturn_30,avgreturn_60,momentum_10,momentum_100,momentum_180,momentum_30,momentum_60,skew_10,skew_100,skew_180,skew_30,skew_60,volatility_10,volatility_100,volatility_180,volatility_30,volatility_60,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7,avgreturn10_shifted,volatility_10_shifted,sharpe_ratio,sharpe_ratio_mean,Y
0,1997-07-25,EQS_0,-0.012419,-0.027867,-0.021799,-0.017527,-0.018084,0.002417,0.000965,0.001501,0.001368,0.002340,0.024167,0.092656,0.258166,0.038305,0.133397,-0.000366,-0.430148,-0.393705,-0.227304,-0.456774,0.011586,0.014642,0.013352,0.012851,0.012667,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,0.000792,0.013111,0.060371,0.010796,1
1,1997-07-25,EQS_1,-0.017053,-0.020400,-0.020233,-0.019890,-0.018676,-0.000581,0.001318,0.001369,0.000985,0.002530,-0.005813,0.127876,0.236827,0.028556,0.146731,-0.149339,-0.215706,-0.026699,0.470290,0.225803,0.011852,0.014635,0.013776,0.014864,0.013114,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,-0.003613,0.010555,-0.342303,0.010796,0
2,1997-07-25,EQS_10,-0.021068,-0.022818,-0.018827,-0.022964,-0.022541,-0.000989,0.000404,0.000697,-0.001385,0.000894,-0.009886,0.039230,0.120511,-0.040176,0.051878,0.579712,-0.090975,-0.050601,0.006137,0.034475,0.015149,0.013122,0.011960,0.012665,0.012172,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,0.003155,0.009809,0.321617,0.010796,1
3,1997-07-25,EQS_2,-0.012701,-0.014182,-0.011899,-0.016148,-0.012140,0.001843,0.001058,0.000796,0.001652,0.002294,0.018427,0.101567,0.136959,0.047898,0.133051,0.059152,-0.064348,-0.019025,0.114231,0.011804,0.011715,0.010250,0.009119,0.011535,0.010179,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,0.002203,0.006017,0.366055,0.010796,1
4,1997-07-25,EQS_3,-0.015363,-0.015846,-0.012966,-0.015688,-0.012114,0.001626,0.001758,0.001219,0.001664,0.002744,0.016257,0.170478,0.210841,0.048260,0.159161,-0.102896,-0.254505,-0.129573,-0.550835,-0.354502,0.011693,0.010184,0.009325,0.010237,0.009328,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,-0.000250,0.005373,-0.046551,0.010796,0


In [21]:
#save to csv
#final_df.to_csv('final_df_m.csv', index=False)

In [22]:
#drop all assets but EQS_0
#final_df = final_df[final_df['asset'] == 'EQ_1']

final_df.head()

,todate,asset,VaR_10,VaR_100,VaR_180,VaR_30,VaR_60,avgreturn_10,avgreturn_100,avgreturn_180,avgreturn_30,avgreturn_60,momentum_10,momentum_100,momentum_180,momentum_30,momentum_60,skew_10,skew_100,skew_180,skew_30,skew_60,volatility_10,volatility_100,volatility_180,volatility_30,volatility_60,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7,avgreturn10_shifted,volatility_10_shifted,sharpe_ratio,sharpe_ratio_mean,Y
0,1997-07-25,EQS_0,-0.012419,-0.027867,-0.021799,-0.017527,-0.018084,0.002417,0.000965,0.001501,0.001368,0.002340,0.024167,0.092656,0.258166,0.038305,0.133397,-0.000366,-0.430148,-0.393705,-0.227304,-0.456774,0.011586,0.014642,0.013352,0.012851,0.012667,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,0.000792,0.013111,0.060371,0.010796,1
1,1997-07-25,EQS_1,-0.017053,-0.020400,-0.020233,-0.019890,-0.018676,-0.000581,0.001318,0.001369,0.000985,0.002530,-0.005813,0.127876,0.236827,0.028556,0.146731,-0.149339,-0.215706,-0.026699,0.470290,0.225803,0.011852,0.014635,0.013776,0.014864,0.013114,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,-0.003613,0.010555,-0.342303,0.010796,0
2,1997-07-25,EQS_10,-0.021068,-0.022818,-0.018827,-0.022964,-0.022541,-0.000989,0.000404,0.000697,-0.001385,0.000894,-0.009886,0.039230,0.120511,-0.040176,0.051878,0.579712,-0.090975,-0.050601,0.006137,0.034475,0.015149,0.013122,0.011960,0.012665,0.012172,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,0.003155,0.009809,0.321617,0.010796,1
3,1997-07-25,EQS_2,-0.012701,-0.014182,-0.011899,-0.016148,-0.012140,0.001843,0.001058,0.000796,0.001652,0.002294,0.018427,0.101567,0.136959,0.047898,0.133051,0.059152,-0.064348,-0.019025,0.114231,0.011804,0.011715,0.010250,0.009119,0.011535,0.010179,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,0.002203,0.006017,0.366055,0.010796,1
4,1997-07-25,EQS_3,-0.015363,-0.015846,-0.012966,-0.015688,-0.012114,0.001626,0.001758,0.001219,0.001664,0.002744,0.016257,0.170478,0.210841,0.048260,0.159161,-0.102896,-0.254505,-0.129573,-0.550835,-0.354502,0.011693,0.010184,0.009325,0.010237,0.009328,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,-0.000250,0.005373,-0.046551,0.010796,0


In [23]:
#drop columns asset, sharpe_ratio, sharpe_ratio_mean
final_df = final_df.drop(columns=['sharpe_ratio', 'sharpe_ratio_mean', 'avgreturn10_shifted', 'volatility_10_shifted'])


In [24]:
final_df.tail()


,todate,asset,VaR_10,VaR_100,VaR_180,VaR_30,VaR_60,avgreturn_10,avgreturn_100,avgreturn_180,avgreturn_30,avgreturn_60,momentum_10,momentum_100,momentum_180,momentum_30,momentum_60,skew_10,skew_100,skew_180,skew_30,skew_60,volatility_10,volatility_100,volatility_180,volatility_30,volatility_60,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7,Y
389506,2024-01-10,FXE_0,-0.012047,-0.013212,-0.013076,-0.012998,-0.008199,-0.001598,0.000015,0.000195,-0.000279,0.000557,-0.014378,0.001432,0.033533,-0.007807,0.031749,-1.218717,-0.285161,-0.275452,-0.331889,-0.178411,0.006815,0.006743,0.007035,0.007163,0.006487,-0.047303,-0.002559,-0.01207,-0.00015,-0.007477,0.010572,-0.013364,-0.016024,0.00123,1
389507,2024-01-10,FXE_1,-0.006983,-0.013866,-0.012628,-0.010463,-0.009131,-0.000153,0.000109,0.000537,0.000686,0.001465,-0.001373,0.010436,0.092964,0.019199,0.083507,0.171425,-0.285339,-0.354310,-0.909760,-0.317357,0.004950,0.007729,0.006928,0.006316,0.006656,-0.047303,-0.002559,-0.01207,-0.00015,-0.007477,0.010572,-0.013364,-0.016024,0.00123,0
389508,2024-01-10,FXE_2,-0.012162,-0.014966,-0.015056,-0.013780,-0.014149,-0.003361,-0.000048,-0.000034,-0.000104,0.000297,-0.030247,-0.004576,-0.005844,-0.002909,0.016941,-0.342480,0.100524,-0.055792,0.302944,0.369697,0.006379,0.009154,0.009521,0.009895,0.009131,-0.047303,-0.002559,-0.01207,-0.00015,-0.007477,0.010572,-0.013364,-0.016024,0.00123,0
389509,2024-01-10,FXE_3,-0.015847,-0.015085,-0.017683,-0.015570,-0.015178,0.003421,0.001248,-0.000098,0.000698,0.002421,0.030786,0.119763,-0.016975,0.019537,0.137991,0.411783,0.510165,0.394898,0.413230,0.385816,0.014255,0.013516,0.013065,0.010623,0.015391,-0.047303,-0.002559,-0.01207,-0.00015,-0.007477,0.010572,-0.013364,-0.016024,0.00123,1
389510,2024-01-10,FXE_4,-0.001067,-0.001675,-0.002661,-0.001665,-0.001516,0.000305,-0.000010,-0.000047,0.000112,0.000061,0.002749,-0.000987,-0.008392,0.003248,0.003580,0.280866,-0.168481,-0.188184,0.973426,0.774549,0.001129,0.001118,0.001508,0.001148,0.001004,-0.047303,-0.002559,-0.01207,-0.00015,-0.007477,0.010572,-0.013364,-0.016024,0.00123,1


In [25]:
final_df.head()

,todate,asset,VaR_10,VaR_100,VaR_180,VaR_30,VaR_60,avgreturn_10,avgreturn_100,avgreturn_180,avgreturn_30,avgreturn_60,momentum_10,momentum_100,momentum_180,momentum_30,momentum_60,skew_10,skew_100,skew_180,skew_30,skew_60,volatility_10,volatility_100,volatility_180,volatility_30,volatility_60,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7,Y
0,1997-07-25,EQS_0,-0.012419,-0.027867,-0.021799,-0.017527,-0.018084,0.002417,0.000965,0.001501,0.001368,0.002340,0.024167,0.092656,0.258166,0.038305,0.133397,-0.000366,-0.430148,-0.393705,-0.227304,-0.456774,0.011586,0.014642,0.013352,0.012851,0.012667,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,1
1,1997-07-25,EQS_1,-0.017053,-0.020400,-0.020233,-0.019890,-0.018676,-0.000581,0.001318,0.001369,0.000985,0.002530,-0.005813,0.127876,0.236827,0.028556,0.146731,-0.149339,-0.215706,-0.026699,0.470290,0.225803,0.011852,0.014635,0.013776,0.014864,0.013114,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,0
2,1997-07-25,EQS_10,-0.021068,-0.022818,-0.018827,-0.022964,-0.022541,-0.000989,0.000404,0.000697,-0.001385,0.000894,-0.009886,0.039230,0.120511,-0.040176,0.051878,0.579712,-0.090975,-0.050601,0.006137,0.034475,0.015149,0.013122,0.011960,0.012665,0.012172,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,1
3,1997-07-25,EQS_2,-0.012701,-0.014182,-0.011899,-0.016148,-0.012140,0.001843,0.001058,0.000796,0.001652,0.002294,0.018427,0.101567,0.136959,0.047898,0.133051,0.059152,-0.064348,-0.019025,0.114231,0.011804,0.011715,0.010250,0.009119,0.011535,0.010179,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,1
4,1997-07-25,EQS_3,-0.015363,-0.015846,-0.012966,-0.015688,-0.012114,0.001626,0.001758,0.001219,0.001664,0.002744,0.016257,0.170478,0.210841,0.048260,0.159161,-0.102896,-0.254505,-0.129573,-0.550835,-0.354502,0.011693,0.010184,0.009325,0.010237,0.009328,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.0,0


In [26]:
#ridge regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score

#final_df_encoded = pd.get_dummies(final_df, columns=['asset'], drop_first=True)

#split the data into train and test
#X = final_df_encoded.drop(columns=['Y', 'todate'])
X = final_df.drop(columns=['Y', 'asset'])

y = final_df['Y']


In [27]:
import pandas as pd

# Convert your date column to datetime format if it's not already
X[dateCol] = pd.to_datetime(X[dateCol])

# Define your split dates
train_end = pd.Timestamp('2008-12-31')
eval_start = pd.Timestamp('2009-01-01')
eval_end = pd.Timestamp('2012-12-31')
test_start = pd.Timestamp('2013-01-01')

X_train = X.loc[X[dateCol] <= train_end]
y_train = y.loc[X[dateCol] <= train_end]

X_eval = X.loc[(X[dateCol] >= eval_start) & (X[dateCol] <= eval_end)]
y_eval = y.loc[(X[dateCol] >= eval_start) & (X[dateCol] <= eval_end)]

X_test = X.loc[X[dateCol] >= test_start]
y_test = y.loc[X[dateCol] >= test_start]

# Optionally, drop the date column if it's not needed for modeling
X_train = X_train.drop(dateCol, axis=1)
X_eval = X_eval.drop(dateCol, axis=1)
X_test = X_test.drop(dateCol, axis=1)


In [28]:
X_train

,VaR_10,VaR_100,VaR_180,VaR_30,VaR_60,avgreturn_10,avgreturn_100,avgreturn_180,avgreturn_30,avgreturn_60,momentum_10,momentum_100,momentum_180,momentum_30,momentum_60,skew_10,skew_100,skew_180,skew_30,skew_60,volatility_10,volatility_100,volatility_180,volatility_30,volatility_60,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
0,-0.012419,-0.027867,-0.021799,-0.017527,-0.018084,0.002417,0.000965,0.001501,0.001368,0.002340,0.024167,0.092656,0.258166,0.038305,0.133397,-0.000366,-0.430148,-0.393705,-0.227304,-0.456774,0.011586,0.014642,0.013352,0.012851,0.012667,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.000000
1,-0.017053,-0.020400,-0.020233,-0.019890,-0.018676,-0.000581,0.001318,0.001369,0.000985,0.002530,-0.005813,0.127876,0.236827,0.028556,0.146731,-0.149339,-0.215706,-0.026699,0.470290,0.225803,0.011852,0.014635,0.013776,0.014864,0.013114,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.000000
2,-0.021068,-0.022818,-0.018827,-0.022964,-0.022541,-0.000989,0.000404,0.000697,-0.001385,0.000894,-0.009886,0.039230,0.120511,-0.040176,0.051878,0.579712,-0.090975,-0.050601,0.006137,0.034475,0.015149,0.013122,0.011960,0.012665,0.012172,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.000000
3,-0.012701,-0.014182,-0.011899,-0.016148,-0.012140,0.001843,0.001058,0.000796,0.001652,0.002294,0.018427,0.101567,0.136959,0.047898,0.133051,0.059152,-0.064348,-0.019025,0.114231,0.011804,0.011715,0.010250,0.009119,0.011535,0.010179,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.000000
4,-0.015363,-0.015846,-0.012966,-0.015688,-0.012114,0.001626,0.001758,0.001219,0.001664,0.002744,0.016257,0.170478,0.210841,0.048260,0.159161,-0.102896,-0.254505,-0.129573,-0.550835,-0.354502,0.011693,0.010184,0.009325,0.010237,0.009328,-0.013365,0.008342,0.006122,0.003334,0.020771,-0.005753,0.006034,-0.022739,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165261,-0.026057,-0.048521,-0.041072,-0.033910,-0.042099,0.003199,-0.003281,-0.001510,0.001982,0.000158,0.028792,-0.318245,-0.264268,0.055506,0.009136,-0.169971,-0.034940,-0.271416,0.318196,-0.112260,0.020383,0.027070,0.020647,0.024192,0.028332,-0.040451,0.016430,0.142686,0.001747,0.031343,0.053923,0.046662,0.041830,0.009915
165262,-0.014690,-0.029624,-0.024843,-0.021265,-0.031078,-0.006411,-0.003035,-0.001432,-0.001527,-0.002094,-0.057702,-0.294427,-0.250539,-0.042764,-0.121433,0.844573,-0.314355,-0.724414,-0.557993,-0.111480,0.007219,0.016706,0.012827,0.015383,0.019118,-0.040451,0.016430,0.142686,0.001747,0.031343,0.053923,0.046662,0.041830,0.009915
165263,-0.013215,-0.034851,-0.028912,-0.016826,-0.043206,0.005297,-0.001498,-0.000813,0.003686,-0.000087,0.047675,-0.145345,-0.142286,0.103197,-0.005069,-0.340003,-0.913806,-1.101603,0.233837,-1.012118,0.013150,0.023399,0.018520,0.017062,0.027660,-0.040451,0.016430,0.142686,0.001747,0.031343,0.053923,0.046662,0.041830,0.009915
165264,-0.023625,-0.019169,-0.014250,-0.016895,-0.022241,-0.005906,-0.001682,-0.001101,0.000078,-0.001510,-0.053154,-0.163142,-0.192737,0.002195,-0.087607,-0.847787,0.101839,-0.152484,-0.223590,0.030763,0.011258,0.011521,0.009140,0.010433,0.013455,-0.040451,0.016430,0.142686,0.001747,0.031343,0.053923,0.046662,0.041830,0.009915


In [29]:
# get latest date in todate column


In [30]:
"""
#initialize the model
model = Ridge()

#fit the model
model.fit(X_train, y_train)

#predict the Y
y_pred = model.predict(X_test)

#round the predictions
y_pred = np.round(y_pred)

#calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ' + str(accuracy))
"""

"""from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score

# Initialize the Ridge Classifier model
ridge_model = RidgeClassifier()

# Train the model on the training set
ridge_model.fit(X_train, y_train)

# Predict on the test set
y_test_pred = ridge_model.predict(X_test)

# Evaluate the model on the test set using accuracy
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)

# Predict on the evaluation set
y_eval_pred = ridge_model.predict(X_eval)

# Evaluate the model on the evaluation set using accuracy
eval_accuracy = accuracy_score(y_eval, y_eval_pred)
print("Evaluation Accuracy:", eval_accuracy)
"""

from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Combine training and evaluation sets for the final model training
X_train_eval = pd.concat([X_train, X_eval])
y_train_eval = pd.concat([y_train, y_eval])


In [31]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error

# Define the parameter grid
param_grid = {'alpha': [0.1, 1.0, 10.0]}

# Initialize the Ridge Classifier and GridSearchCV
ridge = RidgeClassifier()
grid_search = GridSearchCV(ridge, param_grid, scoring='accuracy', cv=5)

# Fit GridSearchCV on the training set
grid_search.fit(X_train, y_train)

# Print the best parameters and the accuracy on the evaluation set
print("Best parameters:", grid_search.best_params_)
print("Best accuracy on evaluation set:", grid_search.best_score_)

# Retrain the model with the best parameters on the combined training and evaluation sets
ridge_best = RidgeClassifier(**grid_search.best_params_)
ridge_best.fit(X_train_eval, y_train_eval)

# Evaluate on the test set
y_test_pred = ridge_best.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test accuracy:", test_accuracy)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)

# Precision
precision = precision_score(y_test, y_test_pred)

# Recall
recall = recall_score(y_test, y_test_pred)

# F1 Score
f1 = f1_score(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)

# Calculate RMSE
rmse = np.sqrt(mse)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

# ROC-AUC Score
# Note: roc_auc_score requires probability scores for its input, not just predictions
# y_test_prob = ridge_best.predict_proba(X_test)[:, 1]
# roc_auc = roc_auc_score(y_test, y_test_prob)

print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Best parameters: {'alpha': 10.0}
Best accuracy on evaluation set: 0.5375818079677973
Test accuracy: 0.5309988937649676
MSE: 0.4690011062350324
RMSE: 0.684836554394574
Confusion Matrix:
[[51131 32681]
 [44480 36230]]
Precision: 0.5257506058539275
Recall: 0.4488910915623838
F1 Score: 0.484290306841954


In [32]:
#xgboost

import xgboost as xgb
from sklearn.metrics import accuracy_score

"""#initialize the model
model = xgb.XGBClassifier()

#fit the model
model.fit(X_train, y_train)

#predict the Y
y_pred = model.predict(X_test)

#calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ' + str(accuracy))
"""
## ovan gav 50.2% accuracy

"""from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Initialize the model
model = xgb.XGBClassifier()

# Define the parameters grid
param_grid = {
    'max_depth': [3, 4, 5],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', n_jobs=-1, cv=3, verbose=2)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

# Predict & evaluate
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)"""







'from sklearn.model_selection import GridSearchCV\nimport xgboost as xgb\n\n# Initialize the model\nmodel = xgb.XGBClassifier()\n\n# Define the parameters grid\nparam_grid = {\n    \'max_depth\': [3, 4, 5],\n    \'n_estimators\': [100, 200, 300],\n    \'learning_rate\': [0.01, 0.1, 0.2]\n}\n\n# Initialize GridSearchCV\ngrid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring=\'accuracy\', n_jobs=-1, cv=3, verbose=2)\n\n# Fit GridSearchCV\ngrid_search.fit(X_train, y_train)\n\n# Best parameters and best score\nprint("Best parameters:", grid_search.best_params_)\nprint("Best score:", grid_search.best_score_)\n\n# Predict & evaluate\ny_pred = grid_search.predict(X_test)\naccuracy = accuracy_score(y_test, y_pred)\nprint(\'Accuracy:\', accuracy)'

In [33]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Initialize the model
xgb_model = xgb.XGBClassifier()

# Define hyperparameters to test
param_grid = {
    'max_depth': [3, 6, 10],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2]
}

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(xgb_model, param_grid, scoring='accuracy', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train, eval_set=[(X_eval, y_eval)], early_stopping_rounds=10, verbose=False)

# Best hyperparameters
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

# Train the model with the best parameters on the combined training and evaluation set
xgb_best = xgb.XGBClassifier(**best_params)
xgb_best.fit(pd.concat([X_train, X_eval]), pd.concat([y_train, y_eval]))




C:\Users\oscar.azrak\AppData\Roaming\Python\Python310\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Best hyperparameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 50}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [34]:
# Evaluate on the test set
y_test_pred = xgb_best.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test accuracy:", test_accuracy)


# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)

# Precision
precision = precision_score(y_test, y_test_pred)

# Recall
recall = recall_score(y_test, y_test_pred)

# F1 Score
f1 = f1_score(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)

# Calculate RMSE
rmse = np.sqrt(mse)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

# ROC-AUC Score
# Note: roc_auc_score requires probability scores for its input, not just predictions
# y_test_prob = ridge_best.predict_proba(X_test)[:, 1]
# roc_auc = roc_auc_score(y_test, y_test_prob)

print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Test accuracy: 0.5306949830417816
MSE: 0.46930501695821836
RMSE: 0.6850584040490404
Confusion Matrix:
[[49072 34740]
 [42471 38239]]
Precision: 0.5239726496663424
Recall: 0.47378267872630403
F1 Score: 0.49761531404329523


In [36]:

"""import lightgbm as lgb
from sklearn.metrics import accuracy_score

#initialize the model
model = lgb.LGBMClassifier()

#fit the model
model.fit(X_train, y_train)

#predict the Y
y_pred = model.predict(X_test)

#calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ' + str(accuracy))"""


## ovan gav 51.2% accuracy
"""
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Initialize the model
model = lgb.LGBMClassifier()

# Define the parameters grid
param_grid = {
    'num_leaves': [31, 50, 100],
    'max_depth': [10, 20, -1],  # -1 means no limit
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', n_jobs=-1, cv=3, verbose=2)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

# Use the best estimator to make predictions
y_pred = grid_search.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)
"""
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import pandas as pd

# Initialize the LightGBM model
lgb_model = lgb.LGBMClassifier()

# Define hyperparameters to test
param_grid = {
    'max_depth': [3, 6, 10],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [7, 15, 31, 63]  
}


# Initialize GridSearchCV
grid_search = GridSearchCV(lgb_model, param_grid, scoring='accuracy', cv=5, n_jobs=-1)

# Fit GridSearchCV on the training data
grid_search.fit(X_train, y_train)

# Best hyperparameters
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

lgb_best = lgb.LGBMClassifier(**best_params)


lgb_best.fit(pd.concat([X_train, X_eval]), pd.concat([y_train, y_eval]), eval_set=[(X_eval, y_eval)])





KeyboardInterrupt: 

In [ ]:
# Evaluate on the test set
y_test_pred = lgb_best.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test accuracy:", test_accuracy)


# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)

# Precision
precision = precision_score(y_test, y_test_pred)

# Recall
recall = recall_score(y_test, y_test_pred)

# F1 Score
f1 = f1_score(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)

# Calculate RMSE
rmse = np.sqrt(mse)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

# ROC-AUC Score
# Note: roc_auc_score requires probability scores for its input, not just predictions
# y_test_prob = ridge_best.predict_proba(X_test)[:, 1]
# roc_auc = roc_auc_score(y_test, y_test_prob)

print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


In [37]:
!{sys.executable} -m pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



     -------------------------------------- 376.9/376.9 MB 7.7 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 49.8 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 42.6 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 65.6 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 23.8 MB/s eta 0:00:00
     --------------------------------------- 26.4/26.4 MB 36.4 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.0 MB/s eta 0:00:00
     ------------------------------------- 413.4/413.4 kB 25.2 MB/s eta 0:00:00
     ---------------------------------------- 3.8/3.8 MB 61.0 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB ? eta 0:00:00
     ---------------------------------------- 133.7/133.7 kB ? eta 0:00:00
     -------------------------------------- 127.8/127.8 kB 7.3 MB/s eta 0:00:00
     ------------------------------------- 220.7/220.7 k

In [38]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

In [44]:
scaler = StandardScaler()
X_train_eval_scaled = scaler.fit_transform(X_train_eval)
X_eval_scaled = scaler.transform(X_eval)
X_test_scaled = scaler.transform(X_test)

In [45]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_eval_scaled.shape[1],)),  # Adjust input_shape as necessary
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Change the output layer according to your task
])

model.compile(optimizer='adam',
              loss='mean_squared_error',  # Change this according to your task
              metrics=['accuracy'])  # Adjust metrics as necessary

C:\Users\oscar.azrak\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [46]:
history = model.fit(
    X_train_eval_scaled, y_train_eval,
    epochs=50,
    batch_size=32,
    validation_data=(X_eval_scaled, y_eval)  # Use X_eval and y_eval directly for validation
)
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


Epoch 1/50
7031/7031 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.5378 - loss: 0.2750 - val_accuracy: 0.5416 - val_loss: 0.2491
Epoch 2/50
7031/7031 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.5573 - loss: 0.2452 - val_accuracy: 0.5636 - val_loss: 0.2440
Epoch 3/50
7031/7031 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.5675 - loss: 0.2429 - val_accuracy: 0.5672 - val_loss: 0.2426
Epoch 4/50
7031/7031 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.5731 - loss: 0.2416 - val_accuracy: 0.5789 - val_loss: 0.2397
Epoch 5/50
7031/7031 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.5794 - loss: 0.2397 - val_accuracy: 0.5846 - val_loss: 0.2371
Epoch 6/50
7031/7031 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.5822 - loss: 0.2388 - val_accuracy: 0.5908 - val_loss: 0.2356
Epoch 7/50
7031/7031 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.5878 - loss: 0.2373 - val_accuracy: 0.5914 - val_loss: 0.2357
Epoch 8/50
7031/7031 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.5925 - loss: 0